# 乳癌のデータセットにて行う

### トレーニングデータを70%、テストデータを30%とする

In [1]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# データを読み込んでトレーニングデータとテストデータに分ける
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, random_state=0)

# トレーニングデータの最大値と最小値を計算
scaler = MinMaxScaler().fit(X_train)

### 数値をMinMaxScalerで標準化し、サポートベクトルマシンで分類結果を示す

In [2]:
# トレーニングデータをスケール変換
X_train_scaled = scaler.transform(X_train)

svm = SVC()
# SVMをスケール変換したデータで訓練
svm.fit(X_train_scaled, y_train)
# テストデータをトレーニングデータの最大値と最小値を用いてスケール変換して、それを用いて評価
X_test_scaled = scaler.transform(X_test)
print("Test score: {:.2f}".format(svm.score(X_test_scaled, y_test)))

Test score: 0.97


### サポートベクトルマシンのハイパーパラメータであるCとgammaをグリッドサーチする

### その際、5クロスバリデーションを行う

### ベストパラメータとベストスコアとテストセットの結果を出す

In [3]:
from sklearn.model_selection import GridSearchCV
# 説明のためのコードであって、実際にこんな風に書いてはだめ
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(SVC(), param_grid = param_grid, cv = 5)
grid.fit(X_train_scaled, y_train)
print("Best cross-validation accuracy best: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Test set accuracy: {:.2f}".format(grid.score(X_test_scaled, y_test)))

Best cross-validation accuracy best: 0.98
Best parameters:  {'C': 1, 'gamma': 1}
Test set accuracy: 0.97


### 上記では、パワーポイントの説明のように情報リーク

## Pipelineの構築

In [4]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])

In [5]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('svm', SVC())])

In [6]:
print("Test score: {:.2f}".format(pipe.score(X_test, y_test)))

Test score: 0.97


### GridSearch内でPipelineの構築

In [7]:
param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

In [8]:
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Best parameters: {}".format(grid.best_params_))

Best cross-validation accuracy: 0.98
Test set score: 0.97
Best parameters: {'svm__C': 1, 'svm__gamma': 1}
